# What students should learn
1. How to build a basic CNN with guard rails.
2. How to take into account dimensions when performing pooling
3. Data augmentation

In [44]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [45]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                            train=False, 
                            transform=transforms.ToTensor(),
                            download = True)
# train_dataset = dsets.MNIST(root='./data', 
#                             train=True, 
#                             transform=transforms.Compose([
#                                 transforms.RandomRotation(degrees = 10),
#                                 transforms.ToTensor(),
#                                 ]),
#                             download=True)

# test_dataset = dsets.MNIST(root='./data', 
#                             train=False, 
#                             transform=transforms.Compose([
#                                 transforms.RandomRotation(degrees = 10),
#                                 transforms.ToTensor(),
#                                 ]),
#                             download = True)

In [46]:
print(train_dataset.train_data.size())
print(train_dataset.train_labels.size())

torch.Size([60000, 28, 28])
torch.Size([60000])


In [47]:
print(test_dataset.test_data.size())
print(test_dataset.test_labels.size())

torch.Size([10000, 28, 28])
torch.Size([10000])


In [48]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [63]:
class CNNModel(nn.Module):
    def __init__(self, in_channels_list = [1, 16], out_channels_list = [16, 32],
                 kernel_size_list = [5, 5], stride_list = [1, 1],
                 padding_list = [2,2], kernel_pool_list = [2, 2],
                 pooling_list = [nn.MaxPool2d, nn.MaxPool2d], activations_list = [nn.ReLU]):
        super(CNNModel, self).__init__()
        argLens = set()
        for argName, arg in locals().items():
            if argName != "activations_list" and argName != "self":
                print(argName)
                print(arg)
                argLens.add(len(arg))
        assert len(argLens) == 1, ("mismatch in lengths of arguments."
                                   "All params must be specified")
#         listLengths = np.array([len()])
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 7 * 7, 10) 

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)

        return(out)

In [36]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()

        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 7 * 7, 10) 

    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2 
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)

        return(out)

In [64]:
model = CNNModel()

argLens
set()
pooling_list
[<class 'torch.nn.modules.pooling.MaxPool2d'>, <class 'torch.nn.modules.pooling.MaxPool2d'>]
kernel_pool_list
[2, 2]
padding_list
[2, 2]
stride_list
[1, 1]
kernel_size_list
[5, 5]
out_channels_list
[16, 32]
in_channels_list
[1, 16]
__class__
<class '__main__.CNNModel'>


TypeError: object of type 'type' has no len()

In [38]:
criterion = nn.CrossEntropyLoss()

In [39]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [40]:
for modelParam in model.parameters():
    print(modelParam.size())
print(model.parameters())

print(len(list(model.parameters())))

torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])
<generator object Module.parameters at 0x7f91a84d0048>
6


In [41]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images
        images = images.requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 50 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images
                images = images.requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

RuntimeError: size mismatch, m1: [100 x 1152], m2: [1568 x 10] at /opt/conda/conda-bld/pytorch_1544174967633/work/aten/src/TH/generic/THTensorMath.cpp:940